### Prestadores desviados

In [163]:
import pandas as pd 
import os

In [345]:
farmacia = pd.read_parquet(f'datos_farmacia/acumulado.gzip', columns = ["N° Prestador","desvios de la media de su sp", "Médico"])
farmacia.rename(columns={"N° Prestador": "Prestador", "Médico": "Razón Social"},inplace=True)
farmacia = farmacia.drop_duplicates(subset='Prestador', keep='last')

cirugias = pd.read_parquet(f'datos_cirugias/acumulado.gzip',columns = ["Prestador", "% en Propias", "Razón social"]) 

consultas = pd.read_parquet(f'datos_consultas/acumulado.gzip', columns = ["Prestador", '2 consultas - participación %', 
                                                                   '3 consultas - participación %',
                                                                   '4 consultas - participación %', 
                                                                   'Más de 4 - participación %',  
                                                                   'Participación %', "Razón social"]) 
desconocimientos = pd.read_parquet(f'datos_desconocimiento/acumulado.gzip',columns = ["Prestador","Q afiliados", "Razón social"]) 
calificaciones = pd.read_parquet(f'datos_calificaciones/acumulado.gzip', columns = ["Prestador", "Promedio total", "Razón social"]) 

imagenes =  pd.DataFrame(pd.read_parquet(f'datos_imagenes/acumulado.gzip', columns = ["N° Prescriptor", "Normalidad", "Razón social"]))
imagenes.rename(columns={"N° Prescriptor": "Prestador"},inplace=True)

quejas = pd.DataFrame(pd.read_parquet(f'datos_quejas/acumulado.gzip', columns = ["Prestador", "Total de quejas prestador", "Razón social"]))
quejas.drop_duplicates(inplace=True)

In [346]:
metricas = [calificaciones, cirugias, consultas, farmacia, imagenes, desconocimientos, quejas]
for i in metricas:
    i.set_index("Prestador",inplace=True)
    i = i.loc[~i.index.duplicated(keep='first')]

In [347]:
desviados_imagenes = []
for i in imagenes.index:
    if imagenes.loc[i,"Normalidad"]  > 0.5:
        desviados_imagenes.append(i)

In [348]:
#dropeo aquellos que tienen todos 0, ya que tienen puras consultas unicas
consultas = consultas.drop(consultas[consultas.isin([0]).iloc[:, 1:].all(axis=1)].index)

desviados_consultas = []
for i in consultas.index:
    if consultas.loc[i,"2 consultas - participación %"]  > 20:
        desviados_consultas.append(i)
    elif consultas.loc[i,"3 consultas - participación %"] > 10:
        desviados_consultas.append(i)
    elif consultas.loc[i,"4 consultas - participación %"]  > 5 :
        desviados_consultas.append(i)
    elif consultas.loc[i,'Más de 4 - participación %']  > 0 :
        desviados_consultas.append(i)
    elif consultas.loc[i,"Participación %"] > 25:
        desviados_consultas.append(i)

In [349]:
len(desviados_consultas)

3221

In [350]:
desviados_calificaciones = []
for i in calificaciones.index:
    if calificaciones.loc[i,"Promedio total"] < 3.5:
        desviados_calificaciones.append(i)

In [351]:
len(desviados_calificaciones)

100

In [352]:
desviados_cirugias = []
for i in cirugias.index:
    if cirugias.loc[i,"% en Propias"] < 0.8:
        desviados_cirugias.append(i)

In [353]:
len(desviados_cirugias)

641

In [354]:
desviados_desconocimiento = []
for i in desconocimiento.index:
    if desconocimiento.loc[i,"Q afiliados"] > 6:
        desviados_desconocimiento.append(i)

In [355]:
len(desviados_desconocimiento)

178

In [356]:
desviados_quejas = []
for i in quejas.index:
    if quejas.loc[i,"Total de quejas prestador"] > 2:
        desviados_quejas.append(i)

In [357]:
len(desviados_quejas)

160

In [360]:
desviados_farmacia = []
for i in farmacia.index:
    if farmacia.loc[i,"desvios de la media de su sp"] > 2:
        desviados_farmacia.append(i)

In [361]:
len(desviados_farmacia)

2302

In [364]:
consultas.rename(columns={"Participación %": "Reconsultas: participación %"},inplace=True)
farmacia.rename(columns={"desvios de la media de su sp" : "desvios de la media de su sp (farmacia)"},inplace=True)
calificaciones.rename(columns={"Promedio total": "Promedio calificaciones"},inplace=True)
desconocimientos.rename(columns= {"Q afiliados": "Cantidad de afiliados que desconocieron"},inplace=True)
cirugias.rename(columns = {"% en Propias": "% de cirgías en centros propios"},inplace=True)
imagenes.rename(columns ={"Normalidad": "Porcentaje de imágenes normales"},inplace=True)

In [ ]:
elegidos = [consultas, calificaciones]

In [ ]:
{consultas:desviados_consultas, calificaciones:desviados_calificaciones}

In [ ]:
desviados = set(desviados_consultas).intersection(set(desviados_calificaciones))

In [365]:
pd.concat([consultas[consultas.index.isin(desviados)], calificaciones[calificaciones.index.isin(desviados)]],axis=1)

,2 consultas - participación %,3 consultas - participación %,4 consultas - participación %,Más de 4 - participación %,Reconsultas: participación %,Promedio calificaciones,Razón social
Prestador,,,,,,,
283,12.12,12.12,0.00,49.49,73.74,2.60,"Brugo Olmedo,Santiago P."
707,32.00,30.00,0.00,0.00,62.00,2.40,"Young,Edgardo"
21788,9.58,10.14,12.39,61.13,93.24,3.20,"Spada,Graciela B."
21873,26.92,28.85,7.69,0.00,63.46,2.80,"Oberberg,Blanca"
22120,30.89,14.63,6.50,4.07,56.10,3.30,"Peppe,Haydeé E."
24584,16.28,6.98,13.95,29.07,66.28,3.20,"Martín,Mónica G."
24675,23.53,22.46,12.83,29.95,88.77,3.00,"Schejtman,Jorge"
24737,23.26,20.93,0.00,37.21,81.40,3.20,"Fernández,Miriam S."
24982,15.38,7.69,20.51,53.85,97.44,3.00,"Rosanova,María Teresa"
